<center>

## ENSE 350 – Math Programming for Software Engineers - Laboratory

# Lab 4: L U Decomposition

### University of Regina
### Faculty of Engineering and Applied Science - Software Systems Engineering
### Lab Instructor: [Adam Tilson](mailto:Adam.Tilson@uregina.ca)

</center>

### Your Name: Aryan Chandra
### Student ID: 200461690

### Objective:

In this lab we will implement LU Decomposition, a method for factorizing a matrix into a Lower triangular and Upper triangular portion, which may be multiplied together to reconstruct the original matrix. These matrices are computed using Gaussian elimination. LU Decompositions are vital to computers performing matrix operations such as solving systems of linear equations, inverting matrices and computing the determinant.

## Part 1: Matrix Class

Complete the implementation of the following class, which represents a matrix. This class should work for a square two dimensional matrix of any size, i.e. the number of rows match the number of columns.

The class should have the following members:
- `self.matrix` - A 2D python list which represents the original 2D matrix provided to the constructor
- `self.upper` - A 2D python list which contains the Upper matrix during and after decomposition
- `self.lower` - A 2D python list which contains the Lower matrix during and after decomposition
- `self.dimensions` - Since the matrix is square, this is the number of cols in the matrix, as well as the number of columns. 


Implement the following functions:

`__init__(self)`: creates a matrix class from a 2D python array. Assume dimensions match. Given.

`decompose(self)`: decompose the matrix into an upper triangular and lower triangular portion, storing these as `self.lower` and `self.upper`. We will not account for the special case where there is a leading zero in a column.

`solve`: given a solution vector, $\vec{b}$ , find the correct values of x using the upper and lower matrices. The entire process is defined by the derivation:
$$ \boldsymbol{A}\vec{x} = \vec{b} $$
$$ \text{let }\boldsymbol{A} = \boldsymbol{LU}$$
$$ \boldsymbol{LU}\vec{x} = \vec{b}$$
$$ \text{let }\boldsymbol{U}\vec{x} = \vec{y}$$
$$ \boldsymbol{L}\vec{y} = \vec{b}$$


`__get_y__`: a helper function for solve, which computes y according to the step:
$$ \boldsymbol{L}\vec{y} = \vec{b}$$
That is, this function performs back-substitution. You should use this in `solve`!
    
`__get_x__`: a helper function for solve, which computes x according to the step:
$$ \boldsymbol{U}\vec{x} = \vec{y}$$
You should use this in `solve`!

`get_determinant`: return the determinant of the matrix

In [26]:
# LU Decomposition
# [A] = [L][U]
# Some good resources:
# https://www.youtube.com/watch?v=14NX5HJxBNY
# http://math.oit.edu/~watermang/math_341/341_ch7/F13_341_book_sec_7-2.pdf
import copy

class Matrix:
    '''
    This function models a Matrix, i.e. a 2D list of numbers.
    >>> matrix = Matrix([[ 7, -2,  1],\
                         [14, -7, -3],\
                         [-7, 11, 18]])
                         
    >>> matrix.print_matrix()
    Matrix:  Original Matrix
    <BLANKLINE>
    rows:    3
    columns: 3
    <BLANKLINE>
        0   1   2 
    0[  7, -2,  1]
    1[ 14, -7, -3]
    2[ -7, 11, 18]
    
    >>> matrix.print_upper()
    Matrix:  Upper Triangular
    <BLANKLINE>
    rows:    3
    columns: 3
    <BLANKLINE>
          0     1     2 
    0[    7,   -2,    1]
    1[  0.0, -3.0, -5.0]
    2[  0.0,  0.0,  4.0]

    >>> matrix.print_lower()
    Matrix:  Lower Triangular
    <BLANKLINE>
    rows:    3
    columns: 3
    <BLANKLINE>
          0     1     2 
    0[    1,    0,    0]
    1[  2.0,    1,    0]
    2[ -1.0, -3.0,    1]
    
    >>> matrix.__get_y__([12, 17, 5])
    [12, -7.0, -4.0]
    
    >>> matrix.__get_x__([12, -7, -4])
    [3.0, 4.0, -1.0]
    
    >>> matrix.solve([12, 17, 5])
    [3.0, 4.0, -1.0]
    
    >>> matrix.get_determinant()
    -84.0
    '''

    def __init__ (self, matrix):
        self.matrix = matrix
        self.dimensions = len(matrix)
        self.decompose()

    def decompose (self):
        '''Decomposes the matrix into upper and lower portions. 
        Use Naive Gaussian Elimination
        '''
        # Your code here. 
        # Save your matrices as self.lower and self.upper
        # Hint: Make 2 deep copies of your original matrix to self.lower 
        # and self.upper, and modify these during decomposition!
        self.upper = copy.deepcopy(self.matrix)
        self.lower = copy.deepcopy(self.matrix)

        for col in range(self.dimensions - 1):
            for row in range(col + 1, self.dimensions):
                multiplier = self.upper[row][col] / self.upper[col][col]
                for k in range(col, self.dimensions):
                    self.upper[row][k] -= multiplier * self.upper[col][k]
                self.lower[row][col] = multiplier
        
        for i in range(self.dimensions):
            self.lower[i][i] = 1

        for row in range(self.dimensions):
            for col in range(self.dimensions):
                if(row < col): 
                    self.lower[row][col] = 0
        pass

    def __get_y__ (self, b):
        # Your code here!
        # Hint: Back substitution using L!
        y = [0] * self.dimensions
        for row in range(self.dimensions):
            y[row] = b[row]
            for j in range(row):
                y[row] -= self.lower[row][j] * y[j]
        return y
        pass
        
    def __get_x__ (self, y):
        # Your code here!
        # Hint: Back substitution using U!
        x = [0] * self.dimensions
        for row in reversed(range(self.dimensions)):
            x[row] = y[row]
            for col in range(row + 1, self.dimensions):
                x[row] -= self.upper[row][col] * x[col]
            x[row] /= self.upper[row][row]
        return x
        pass
        
    def solve (self, b):
        '''Assuming your matrix represents a system of linear equations, solve
        this system using your lower and upper triangular matrices for a 
        right-hand-side vector, b'''
        y = self.__get_y__(b)
        x = self.__get_x__(y)
        return x

    def get_determinant (self):
        # Your code here!
        answer = 1
        for row in range(self.dimensions):
            for col in range(self.dimensions):
                if row == col:
                    answer *= self.upper[row][col]
        return answer
        pass

    ## The following code is used to print the matrix. You may wish to make use
    ## of these functions to help when debugging!
    ## You do not need to understand this code.
    ## If you change it, test cases will break!

    def print_matrix (self):
        Matrix.__print_helper__(self.matrix, "Original Matrix")
        
    def print_upper (self):
        Matrix.__print_helper__(self.upper, "Upper Triangular")
    
    def print_lower (self):
        Matrix.__print_helper__(self.lower, "Lower Triangular")
    
    def __print_helper__ (matrix, name):
        # prints a formatted matrix and dimensions
        
        print ("Matrix: ", name)
        print ()
        print ("rows:   ", len(matrix))
        print ("columns:", len(matrix[0]))
        print ()
        
        # compute spacing variables
        max_width = 0
        for i in matrix:
            for j in i:
                len_number_string = len(f'{j}')
                if len_number_string > max_width:
                    max_width = len_number_string              
        
        first_row_padding = len(f'{len(matrix)}')
        max_width += 2
        
        # column indices:
        print (f'{" ": >{first_row_padding+1}}', end="")
        for j in range (len(matrix[0])):
            print (f'{str(j) + " ": >{max_width}}', end="")  
        print()
        
        # print row by row
        for i in range(len(matrix)):
            first = True
            
            # row number...
            print (f'{str(i):>{first_row_padding}}', end="")  
            
            for j in matrix[i][:-1]:
                if first:
                    first = False
                    print ("[" + f'{str(j) + ",":>{max_width}}', end="") 
                else:
                    print (f'{str(j) + ",":>{max_width}}' , end="")
            # last row
            print (f'{str (matrix[i][-1])+"]":>{max_width}}')

import doctest
doctest.testmod()

TestResults(failed=0, attempted=8)

## Part 2: Application

Using your code from part 1, solve the following system of linear equations.

$$ x_1 + 2x_2 + x_3 - x_4 = 5 $$
$$ 3x_1 + 2x_2 + 4x_3 + 4x_4 = 16 $$
$$ 4x_1 + 4x_2 + 3x_3 + 4x_4 = 22 $$
$$ 2x_1 + x_3 +5x_4 = 15 $$


In [33]:
# Your code here!
matrix2 = Matrix([[1, 2, 1, -1],\
                    [3, 2, 4, 4],\
                    [4, 4, 3, 4],\
                    [2, 0, 1, 5]])

matrix2.print_matrix()
print()
matrix2.print_upper()
print()
matrix2.print_lower()
print()
print("values of x1, x2, x3, and x4 are:",matrix2.solve([5, 16, 22, 15]))

Matrix:  Original Matrix

rows:    4
columns: 4

    0   1   2   3 
0[  1,  2,  1, -1]
1[  3,  2,  4,  4]
2[  4,  4,  3,  4]
3[  2,  0,  1,  5]

Matrix:  Upper Triangular

rows:    4
columns: 4

      0     1     2     3 
0[    1,    2,    1,   -1]
1[  0.0, -4.0,  1.0,  7.0]
2[  0.0,  0.0, -2.0,  1.0]
3[  0.0,  0.0,  0.0, -1.0]

Matrix:  Lower Triangular

rows:    4
columns: 4

     0    1    2    3 
0[   1,   0,   0,   0]
1[ 3.0,   1,   0,   0]
2[ 4.0, 1.0,   1,   0]
3[ 2.0, 1.0, 1.0,   1]

values of x1, x2, x3, and x4 are: [16.0, -6.0, -2.0, -3.0]


Solve the same system again, this time using the following right-hand-side vector. Do not decompose the matrix again!
$$ \vec{b} = \begin{bmatrix}
           23 \\
           30 \\
           11 \\
           5 
         \end{bmatrix} $$

In [34]:
# Your code here!
print("values of x1, x2, x3, and x4 are:",matrix2.solve([23, 30, 11, 5]))

values of x1, x2, x3, and x4 are: [102.0, -60.0, 1.0, -40.0]
